In [1]:
%load_ext autoreload
%autoreload 2

from test import test_model
from src.config import config
from src.data.generate_data import generate_movingai

''' Ensemble (list of tuples made of 5 elements)
use_stepinfer_method, astar_type, active_agent_radius, use_aep, aep_astar_type
- use_stepinfer_method: True or False
- astar_type: 1: All agents are treated as obstacles, 2: Treat Finished agents as obstacles, 3: Not regarding any agent as obstacles, None: Not use astar
- active_agent_radius: only if astar_type is not None. The radius of the active agent to be considered for astar. This parameter is only for hybrid guidance, NOT for AEP.
- use_aep: True or False
- aep_astar_type: 1: All agents are treated as obstacles, 2: Treat Finished agents as obstacles, 3: Not regarding any agent as obstacles, None: Not use astar
'''

config.ensemble = [
    (False, None, None, False,None),]

config.test_folder = "./test_set"
config.save_map_config = True
config.save_positions = True
config.data_mode = "movingai"

# config.action_dim=2
config.test_env_settings = (
                    # ("den312d", 4),
                    # ("den312d", 8),
                    # ("den312d", 16),
                    # ("den312d", 32),
                    # ("den312d", 64),
                    ("warehouse", 4),
                    ("warehouse", 8),
                    # ("warehouse", 16),
                    # ("warehouse", 32),
                    # ("warehouse", 64),
                    )

Setting random seed to 42


In [2]:
generate_movingai(20, config.test_env_settings)

Generating instances for warehouse with 4 agents ...


100%|██████████| 20/20 [00:00<00:00, 150.88it/s]


Generating instances for warehouse with 8 agents ...


100%|██████████| 20/20 [00:00<00:00, 146.19it/s]


In [3]:
import rich
from omegaconf import OmegaConf

rich.print(OmegaConf.to_yaml(config))


action_dim: 5
active_agent_radius: 3
actor_update_steps: 200
aep_astar_type: 2
aep_distance_threshold: null
aep_use_q_value_only: false
astar_type: 2
batch_size: 128
block_rate_threshold: 0.5
buffer_capacity: 262144
burn_in_steps: 20
chunk_capacity: 64
cl_history_size: 100
cnn_channel: 128
congestion_dispair: 3
congestion_expend: 1
congestion_radius: 2
density: 0.3
ensemble:
- - false
  - null
  - null
  - false
  - null
forward_steps: 2
gamma: 0.99
grad_norm_dqn: 40
hidden_dim: 256
init_env_settings:
- 1
- 10
learning_starts: 50000
log_interval: 10
lr: 0.0002
lr_scheduler_gamma: 0.5
lr_scheduler_milestones:
- 40000
- 80000
max_comm_agents: 3
max_episode_length: 256
max_episode_length_80: 386
max_episode_warehouse: 512
max_map_lenght: 40
max_num_agents: 16
model_target: src.models.model_V0.Network
name: MyModel_V0
num_actors: 4
num_test_cases: 100
obs_radius: 4
obs_shape:
- 8
- 9
- 9
open_closed_goal: false
pass_rate: 0.9
prioritized_replay_alpha: 0.6
prioritized_replay_beta: 0.4
project: FinalTest
reward_fn:
  move: -0.055
  stay_on_goal: 0
  stay_off_goal: -0.055
  collision: -0.5
  finish: 3
  goin_congestion: -0.1
  stay_congestion: -0.005
  has_go: -0.003
run_id: 20240426-100824
save_interval: 5000
save_map_config: true
save_path: ./saved_models/model_V0
save_positions: true
selective_comm: true
target_network_update_freq: 1750
test_data_mode: normal
test_env_settings:
- - warehouse
  - 4
- - warehouse
  - 8
test_folder: ./test_set
test_seed: 0
test_timeout: 300
training_steps: 200000
use_aep: false
use_stepinfer_method: false
use_validation: true
use_wandb: true
use_wandb_test: false
val_device: cpu
val_interval: 5000
val_pool_parallel_cores: 16
val_test_set:
- - 40
  - 16
  - 0.3
- - 40
  - 32
  - 0.3
- - 40
  - 64
  - 0.3
weight_decay: 0
data_mode: movingai

In [ ]:
results = test_model('10000', config=config)


----------test model MyModel_V0@10000----------
test set: warehouse 4 agents


100%|██████████| 20/20 [00:51<00:00,  2.57s/it]


success rate: 65.00%
average step: 271.95
communication times: 46.1
average arrived agents : 3.45 
Time taken for test set:  51.32
Saving results in ./results/warehouse_4agents/MyModel_V0_10000.pkl
test set: warehouse 8 agents


  0%|          | 0/20 [00:00<?, ?it/s]

## Create visualization

First, we need to collect the results. Let's choose the last case

In [35]:
print(results.keys())

dict_keys([['warehouse', 4], ['warehouse', 8]])


In [47]:
IDX = 9# take one solution

key = list(results.keys())[1]
print(key)
res = results[key]

all_map = res["map"][IDX]
map_, init_pos, goals = all_map[0], all_map[1], all_map[2]
positions = res["best_pos"][IDX] # for ensemble, it is a list of positions
print("Makespan: ", len(positions))

['warehouse', 8]
Makespan:  512


Note: Remember to install ffmpeg!

example:
```bash
conda install -c conda-forge ffmpeg
```

In [48]:
from src.utils.video import create_video

FILENAME = "test.mp4"
create_video(map_, init_pos=init_pos, goal_pos=goals, positions=positions, filename=FILENAME)

  0%|          | 0/512 [00:00<?, ?it/s]

In [10]:
from IPython.display import Video

Video(FILENAME, embed=True)